In [4]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_runtime import (
    QiskitRuntimeService,
    Session,
    Options,
    EstimatorResult,
    SamplerResult,
)

from quantum_serverless import QuantumServerless
from quantum_serverless.library import ParallelEstimator, ParallelSampler

In [2]:
# make sure that your IBM cridential is setup

In [3]:
serverless = QuantumServerless()

serverless.clusters()

[<Cluster: Local (all cores)>]

In [5]:
service = QiskitRuntimeService()
options = Options(optimization_level=1, backend="ibmq_qasm_simulator")
options.execution.shots = 10

In [6]:
# esimator

psi1 = RealAmplitudes(num_qubits=2, reps=2)
obs = SparsePauliOp.from_list([("II", 1), ("IZ", 2), ("XI", 3)])
theta1 = [0, 1, 1, 2, 3, 5]

In [7]:
with Session(service=service) as session, serverless.context():
    estimator = ParallelEstimator(session=session, options=options)

    estimator.add(circuits=[psi1], observables=[obs], parameter_values=[theta1])
    estimator.add(circuits=[psi1], observables=[obs], parameter_values=[theta1])
    estimator.add(circuits=[psi1], observables=[obs], parameter_values=[theta1])

    results = estimator.run_all()

results

[EstimatorResult(values=array([1.8]), metadata=[{'variance': 4.679999999999998, 'shots': 10}]),
 EstimatorResult(values=array([1.4]), metadata=[{'variance': 10.92, 'shots': 10}]),
 EstimatorResult(values=array([1.4]), metadata=[{'variance': 10.92, 'shots': 10}])]

In [10]:
# sampler

bell = QuantumCircuit(2)
bell.h(0)
bell.cx(0, 1)
bell.measure_all()

bell.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

In [11]:
with Session(service=service) as session, serverless.context():
    sampler = ParallelSampler(session=session, options=options)

    sampler.add(circuits=bell)
    sampler.add(circuits=bell)
    sampler.add(circuits=bell)
    sampler.add(circuits=bell)

    results = sampler.run_all()
    
results

[SamplerResult(quasi_dists=[{'00': 0.6, '11': 0.4}], metadata=[{'header_metadata': {'circuit_digest': 'fd61714c10a071d1e9b86e8482a8f5274ddb4445c540674201d99b53f97d97fe'}, 'shots': 10}]),
 SamplerResult(quasi_dists=[{'11': 0.4, '00': 0.6}], metadata=[{'header_metadata': {'circuit_digest': 'fd61714c10a071d1e9b86e8482a8f5274ddb4445c540674201d99b53f97d97fe'}, 'shots': 10}]),
 SamplerResult(quasi_dists=[{'11': 0.4, '00': 0.6}], metadata=[{'header_metadata': {'circuit_digest': 'fd61714c10a071d1e9b86e8482a8f5274ddb4445c540674201d99b53f97d97fe'}, 'shots': 10}]),
 SamplerResult(quasi_dists=[{'11': 0.8, '00': 0.2}], metadata=[{'header_metadata': {'circuit_digest': 'fd61714c10a071d1e9b86e8482a8f5274ddb4445c540674201d99b53f97d97fe'}, 'shots': 10}])]